In [ ]:
from matplotlib import pyplot as plt
import cv2
import numpy as np

In [ ]:
!gdown --id 13f_vZ51V4hUO-kuvXnqlb4Nq6fgdxHZO &> /dev/null
!gdown --id 1AaFNIVGPG8SkcmKduvyMOrwuEJWIyVtt &> /dev/null
!python setup.py

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from utils import *

In [ ]:
!gdown --id 1rPvkpWUuXvPCx-vhSGE9lD6KK4ujfQrY

In [ ]:
!unzip images.zip

unzip:  cannot find or open images.zip, images.zip.zip or images.zip.ZIP.


In [ ]:
frames=[]#Array for detected and embeded frames
framess=[]#Array for all frames
plans=[[]]#Array for plans
cap= cv2.VideoCapture('/content/drive/MyDrive/Friends/f1080_1920.mp4')
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('Frame count:', frame_count)
f=1
c1=[0]
cap.set(cv2.CAP_PROP_POS_FRAMES, f)
while(cap.isOpened() and f<2000):
    ret, frame = cap.read()
    if ret == False:
      break
    bboxesf, lmarksf, cropsf, embeddingsf = detect_and_embed(frame)
    if (len(bboxesf)<=0):
      f+=10
      cap.set(cv2.CAP_PROP_POS_FRAMES, f)
    else:


      #Plan changing
      plans[-1].append(frame)
      image = frame.copy()
      gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
      histogram = cv2.calcHist([gray_image], [0],
                               None, [256], [0, 256])
      cap.set(cv2.CAP_PROP_POS_FRAMES, f+10)
      ret, frame = cap.read()
      if ret == False:
        break
      image1 = frame.copy() 
      gray_image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY) 
      histogram1 = cv2.calcHist([gray_image1], [0],
                                None, [256], [0, 256]) 
      c1=0
      i = 0
      while i<len(histogram) and i<len(histogram1): 
          c1+=(histogram[i]-histogram1[i])**2
          i+= 1
      c1 = c1**(1 / 2) 
      if c1[0]>65000:
        plans.append([])




      #Frame adding
      cap.set(cv2.CAP_PROP_POS_FRAMES, f)
      ret, frame = cap.read()
      if ret == False:
        break
      bboxesf, lmarksf, cropsf, embeddingsf = detect_and_embed(frame)
      framess.append(frame)
      frames.append([bboxesf,lmarksf,cropsf,embeddingsf])
      f+=10
      cap.set(cv2.CAP_PROP_POS_FRAMES, f)



height=framess[0].shape[0]
width=framess[0].shape[1]






"""for i in range(len(plans)):
  print("#######################################")
  for j in range(len(plans[i])):
    cv2_imshow(plans[i][j])
"""


chars=[]
charst=[]
ww=-1
color=[(0,255,0),(255,0,0),(0,0,255),(255,255,0),(255,0,255),(0,255,255),(255,255,255),
       (128,64,0),(64,128,0),(0,64,128),(0,128,64),(64,0,128),(128,0,64),(0,0,0),
       (220,64,64),(64,220,64),(64,64,220),(220,128,220),(150,20,110),(125,250,64),(55,55,55)]
name=["Target1","Target2","Target3","Target4","Target5","Target6","Target7",
      "Target8","Target9","Target10","Target11","Target12","Target13","Target14",
      "Target15","Target16","Target17","Target18","Target19","Target20","Target21"]
for t in range(len(plans)):
  chars.append([])
  charst.append([])
  for i in range(len(plans[t])):
    ww+=1
    for j in range(len(frames[ww][0])):
      flag=0
      for k in range(len(chars[t])):
        flagak=0
        for l in range(len(chars[t])):
          if (chars[t][l][0][0]-40<frames[ww][0][j][0] and
              chars[t][l][0][0]+40>frames[ww][0][j][0] and
              chars[t][l][0][1]-40<frames[ww][0][j][1] and
              chars[t][l][0][1]+40>frames[ww][0][j][1]):
            flagak+=1
            if flagak==2:
              print("+++++++++++++++++++++++++++++++++++++++")
              break
        if flagak<2:
          if (cosine_similarity(chars[t][k][3], frames[ww][3][j])[0,0]>0.7 or
          (chars[t][k][0][0]-40<frames[ww][0][j][0] and
          chars[t][k][0][0]+40>frames[ww][0][j][0] and
          chars[t][k][0][1]-40<frames[ww][0][j][1] and
          chars[t][k][0][1]+40>frames[ww][0][j][1])):
            flag=1
            chars[t][k][0]=frames[ww][0][j].copy()
            chars[t][k][1]=frames[ww][1][j].copy()
            chars[t][k][2]=frames[ww][2][j].copy()
            chars[t][k][3]=frames[ww][3][j].copy()
            charst[t][k].append([ww,j])
            break
        else:
          if (cosine_similarity(chars[t][k][3], frames[ww][3][j])[0,0]>0.4):
            flag=1
            chars[t][k][0]=frames[ww][0][j].copy()
            chars[t][k][1]=frames[ww][1][j].copy()
            chars[t][k][2]=frames[ww][2][j].copy()
            chars[t][k][3]=frames[ww][3][j].copy()
            charst[t][k].append([ww,j])
            break
      if flag==0:
        chars[t].append([frames[ww][0][j],frames[ww][1][j],frames[ww][2][j],frames[ww][3][j]])
        charst[t].append([[ww,j]])
    #cv2_imshow(plans[t][i])





for t in range(len(charst)-1):
  charsel=[]
  for i in range(len(charst[0])):
    if (len(charst[0][i])>=7):
      numi=7
    else:
      numi=len(charst[0][i])
    for j in range(len(charst[t+1])):
      if (len(charst[t+1][j])>=7):
        numj=7
      else:
        numj=len(charst[t+1][j])
      sum=0
      for k in range(numi):
        for l in range(numj):
          sum+=cosine_similarity(frames[charst[0][i][k*(len(charst[0][i])//numi)][0]][3][charst[0][i][k*(len(charst[0][i])//numi)][1]],
                                 frames[charst[t+1][j][l*(len(charst[t+1][j])//numj)][0]][3][charst[t+1][j][l*(len(charst[t+1][j])//numj)][1]])[0,0]
      sum/=(numi*numj)
      print(sum)
      if sum>0.3:
        if j not in charsel:
          charst[0][i]+=charst[t+1][j]
          charsel.append(j)
          break
  for i in range(len(charst[t+1])):
    if i not in charsel:
      charst[0].append(charst[t+1][i])



for i in range(len(charst[0])):
  print(charst[0][i])
for i in range(len(charst[0])):
  for j in range(len(charst[0])-1):
    if(len(charst[0][j])>len(charst[0][j+1])):
      t=charst[0][j].copy()
      charst[0][j]=charst[0][j+1].copy()
      charst[0][j+1]=t.copy()
for i in range(len(charst[0])):
  print(charst[0][i])


charsel=[]
for i in range(len(charst[0])):
  if (len(charst[0][i])>=10):
    numi=10
  else:
    numi=len(charst[0][i])
  for j in range(i+1,len(charst[0])):
    w=0
    if (len(charst[0][j])>=10):
      numj=10
    else:
      numj=len(charst[0][j])
    for k in range(numi):
      for l in range(numj):
        if cosine_similarity(frames[charst[0][i][k*(len(charst[0][i])//numi)][0]][3][charst[0][i][k*(len(charst[0][i])//numi)][1]],
          frames[charst[0][j][l*(len(charst[0][j])//numj)][0]][3][charst[0][j][l*(len(charst[0][j])//numj)][1]])[0,0]>0.3:
          w+=1
    if (w/(numi*numj)>0.2):
      charsel.append(i)
      charst[0][j]+=charst[0][i]
      break

print(charsel)
newchar=[]
for i in range(len(charst[0])):
  if i not in charsel:
    newchar.append(charst[0][i])

print("***********************************")
w=0
for i in range(len(newchar)):
  if (len(newchar[w])<5):
    newchar.pop(w)
    w-=1
  w+=1
for i in range(len(newchar)):
  print(newchar[i])

time=[]
timer=[]
for i in range(len(newchar)):
  time.append(len(newchar[i]))
  timer.append([0,0])

charst[0]=newchar.copy()
charst[0].sort()
        


allchars=[]
for i in range(len(charst[0])):
  for j in range(len(charst[0][i])):
    allchars.append(charst[0][i][j])
    allchars[-1].append(i)


allchars.sort()
for i in range(len(allchars)):
  cv2.rectangle(framess[allchars[i][0]], (frames[allchars[i][0]][0][allchars[i][1]][0], frames[allchars[i][0]][0][allchars[i][1]][1]),
               (frames[allchars[i][0]][0][allchars[i][1]][2], frames[allchars[i][0]][0][allchars[i][1]][3]), color[allchars[i][2]], 2)
  cv2.putText(framess[allchars[i][0]], name[allchars[i][2]] +" "+str(timer[allchars[i][2]][0])+"."+ str (timer[allchars[i][2]][1]),
              (frames[allchars[i][0]][0][allchars[i][1]][0], int (frames[allchars[i][0]][0][allchars[i][1]][1]-12)),
              cv2.FONT_HERSHEY_SIMPLEX,0.6, color[allchars[i][2]], 2, cv2.LINE_AA)
  timer[allchars[i][2]][1]+=10
  if timer[allchars[i][2]][1]==30:
    timer[allchars[i][2]][0]+=1
    timer[allchars[i][2]][1]=0

out = cv2.VideoWriter('video.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 3, (width,height))
for i in range(len(framess)):
  out.write(framess[i])
out.release()
cap.release()
cv2.destroyAllWindows

Frame count: 4274
121809.65
9627.162
9002.956
10487.764
8409.183
8581.218
12614.152
9940.556
10051.452
11168.677
7449.1655
6456.8467
105169.52
23307.229
11883.6455
286640.97
7908.737
10003.993
4485.6055
4893.6274
9169.925
284562.2
22137.068
35126.457
47418.97
46248.902
36594.4
21321.27
99633.51
12474.918
12219.954
13997.657
9412.423
11645.824
43905.52
64876.68
74970.68
17354.102
20273.29
15471.429
19335.91
18701.623
150590.47
35294.43
43662.93
48869.363
16979.355
19334.074
8264.721
13850.873
23579.797
35314.543
38738.69
22831.809
14767.039
251544.73
8177.9033
7527.592
6015.903
95568.375
29113.904
22328.71
20630.541
19329.752
32670.713
19903.2
16280.842
15884.451
26993.57
96352.64
64332.586
65909.14
24924.191
47335.87
86533.77
6010.8228
6328.6167
5494.4526
5813.304
92134.51
11404.736
7580.1206
11330.793
9800.306
9112.829
38881.367
39870.797
25039.729
18675.824
18900.125
24210.963
33127.176
22971.031
22533.406
23112.105
24138.494
17030.914
13772.486
25380.197
20972.334
16520.26
99535.9
1

<function destroyAllWindows>

In [ ]:
img1 = cv2.imread("/content/drive/MyDrive/Colab_Notebooks/images/1.jpg")
bboxes1, lmarks1, crops1, embeddings1 = detect_and_embed(img1)
print(bboxes1[0])

In [ ]:
a=[-1,7,5,2]
a.sort()
print(a)